# **Introduction**

This is a simple csv to SQLite converter.<br>
It was initially thought in the context of a boot camp at Masterschool, where students were eager to train their SQL skills.<br>
We thus wanted to provide a hands-on solution for students to import a .csv of their liking to convert it in a SQLite database for data exploration.<br>
<br>
As such, this code allows one to upload a .csv; normalize the column's names; and output a .db file for use in an application of choice.

In [ ]:
import os
import shutil
import pandas as pd
import sqlite3 as sql3
import sqlalchemy as sa

In [ ]:
# Create database for SQLite

# Replace name in 'database_name' variable for naming your db
database_name = 'database_test'

file_path = (f'{database_name}.db')
if not os.path.exists(file_path):
  open((file_path), 'w')
else:
  print('Database already exists.')

In [ ]:
# Connexion to database

connection = sql3.connect(file_path)
cursor = connection.cursor()

In [ ]:
# Scan for .csv in main folder
# Isolate .csv file(s)

csv_files = [f for f in os.listdir() if '.csv' in f]
print(csv_files)

In [ ]:
# Make a new directory
# Name the directory:
dataset_dir = 'dataset'

# Bash command to create the directory:
try:
  os.mkdir(dataset_dir)
except:
  pass

In [ ]:
# Move the .csv file(s) in the new directory (dataset_dir var):

for csv in csv_files:
  shutil.move(csv, dataset_dir+'/'+csv)
  print(f"'{csv}' moved in '{dataset_dir}/{csv}'")


In [ ]:
# Create DataFrame out of .csv files:
data_path = os.getcwd()+'/'+dataset_dir+'/'

# df var is a dictionnary that uses the names of the .csv files as keys
df = {}
for file in csv_files:
  try:
    df[file] = pd.read_csv(data_path+file)
  except UnicodeDecodeError:
    df[file] = pd.read_csv(data_path+file, encoding = 'ISO-8859-1')
  print(file)

In [ ]:
# Connexion to database
connection = sql3.connect(file_path)
cursor = connection.cursor()


# Clean table and column names
for k in csv_files:

  dataframe = df[k]

  # Clean table's names
  clean_table_name = k.lower().replace(' ', '_').replace('?', '') \
                     .replace('-', '_').replace(r'/', '_').replace('\\', '_') \
                     .replace('%', '').replace(')', '').replace(r'(','') \
                     .replace('$', '').replace('.csv', '')
  print(f"Table's name: {clean_table_name}")


  # Since digit at the beginning of a column's name is a cause for error in SQLite,
  # we start by removing and placing them at the end of the column's name:
  col_names = []
  for name in dataframe.columns:
    digits = ''
    new_col_name = ''
    for x in name:
      if x.isdigit():
        digits += x
      else:
        new_col_name += x
    col_names.append(new_col_name + '_' + digits)

  col_names_corrected = []
  for x in col_names:
    if x.endswith('_'):
      col_names_corrected.append(x[:-1])
    elif x.startswith('_'):
      col_names_corrected.append(x[1:])
    elif x.startswith(' '):
      col_names_corrected.append(x[1:])
    else:
      col_names_corrected.append(x)

  dataframe.columns = col_names_corrected


  # Clean column's names:
  dataframe.columns =  [x.lower().replace(' ', '_').replace('?', '') \
                     .replace('-', '_').replace(r'/', '_').replace('\\', '_') \
                     .replace('%', '').replace(')', '').replace(r'(','') \
                    .replace('$', '') for x in dataframe.columns]
  print(dataframe.columns)


  # Converts Pandas dtype into SQL dtype
  convert_type = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'int',
    'datetime64' : 'timestamp',
    'timedelta64[ns]' : 'varchar'
    }


  # Table schema
  col_str = ', '.join((f'{n} {d}') for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(convert_type)))


  # Drop table if already exists
  cursor.execute(f"DROP TABLE IF EXISTS {clean_table_name}")

  # Create table
  cursor.execute(f"CREATE TABLE {clean_table_name}({col_str})")


  # Insert data into table
  # Export dataframe as csv
  dataframe.to_csv(k, header = dataframe.columns, index=False, encoding='UTF-8')

  # Open created csv file
  my_file = open(k)
  print(f"'{k}' file opened in memory'")

  # Insert dataframe into table
  dataframe.to_sql(clean_table_name, connection, if_exists='replace', index = False)

  print(f'table {clean_table_name} successfully imported to database \n')

print('All table(s) have been successfully imported to database')

In [ ]:
# Close connection
cursor.close()

In [ ]:
# If you want to work on your database from colab, reconnect to the database first

connection = sql3.connect(file_path)
cursor = connection.cursor()

In [ ]:
# Type your SQL querry between the quotation marks

query = '''


'''

pd.read_sql(query, connection)